In [16]:
import pandas as pd
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("SQLite.db")
consumptions = pd.read_sql_query("SELECT * from consumptions", con)
mapping = pd.read_sql_query("SELECT * from feed_station_mapping", con)

# Validate that you have 521974 records in the consumptions dataset, 
# and 1125 records in the feed_station_mapping dataset.
print(len(consumptions.index))
print(len(mapping.index))

521974
1125


In [46]:
# - How many unique stations are there in the data set?

df1 = pd.read_sql_query("""select count(distinct f.station_id) as unique_stations
                       from consumptions c 
                       left join feed_station_mapping f on f.feed_id = c.feed_id;""", 
                       con)
print(df1)

   unique_stations
0             1122


In [44]:
#- How many unique artists are there in the data set?

df2 = pd.read_sql_query("""select count(distinct c.artist) as unique_artists
                        from consumptions c;""", 
                       con)
print(df2)

   unique_artists
0               8


In [43]:
#- How many unique tracks are there in the data set?
df3 = pd.read_sql_query("""select 
                            count(distinct (artist || ' - ' || track)) as unique_tracks
                            from consumptions;""", 
                       con)
print(df3)

   unique_tracks
0              8


In [47]:
#- Who are the top 3 artists based number of consumption events per artist?
df4 = pd.read_sql_query("""select artist, count(stamp) as consumption_events
                        from consumptions
                        group by artist
                        order by count(stamp) desc
                        limit 3;""",
                        con)

print(df4)

      artist  consumption_events
0  artist_03              180794
1  artist_07              172805
2  artist_02               44145


In [54]:
#- Which are the top 3 tracks?
df5 = pd.read_sql_query("""
select (artist || ' - ' || track) as unique_tracks, 
       count(stamp) as consumption_events
from consumptions
group by track
order by count(stamp) desc
limit 3;""",con)
print(df5)

         unique_tracks  consumption_events
0  artist_03 - Song_05              180794
1  artist_07 - Song_04              172805
2  artist_02 - Song_01               44145


In [57]:
# REPORT

con.execute("""
CREATE TABLE mart(
    stamp timestamp,
    concat_track varchar,
    track varchar,
    artist varchar,
    feed_id varchar,
    feed_name varchar,
    fsm.station_id varchar,
   PRIMARY KEY (stamp),
   FOREIGN KEY (stamp, feed_id)
      REFERENCES consumptions (stamp)
         ON DELETE CASCADE 
         ON UPDATE NO ACTION,
   FOREIGN KEY (feed_id)
      REFERENCES groups (feed_station_mapping)
         ON DELETE CASCADE 
         ON UPDATE NO ACTION
);""")

con.execute("""
INSERT INTO mart select c.stamp,
                        c.track,
                        c.artist,
                        c.feed_id,
                        c.feed_name,
                        fsm.station_id
                 from consumptions c
                 left join feed_station_mapping fsm on c.feed_id = fsm.feed_id
);""")



In [49]:
# Top 2 track per feed

df6 = pd.read_sql_query("""
select feed_id,
       track,
       listens,
       rank_id
from (select distinct feed_id,
       track,
       listens,
       rank() OVER (PARTITION BY feed_id ORDER BY listens DESC) as rank_id
       from (select distinct a.feed_id,
                             a.track,
                             count(stamp) OVER (PARTITION BY feed_id, track) as listens
             from (select c.*, fsm.station_id
                   from consumptions c
                   left join feed_station_mapping fsm on c.feed_id = fsm.feed_id) as a
        ) as b
group by feed_id, track
order by feed_id)
where rank_id <3;""",con)

print(df6)

          feed_id    track  listens  rank_id
0      feed_id_01  Song_05      127        1
1      feed_id_02  Song_01       68        1
2      feed_id_02  Song_04       32        2
3      feed_id_03  Song_05      201        1
4      feed_id_03  Song_04        5        2
...           ...      ...      ...      ...
2416  feed_id_996  Song_04        4        2
2417  feed_id_997  Song_07      612        1
2418  feed_id_998  Song_04      779        1
2419  feed_id_998  Song_05      117        2
2420  feed_id_999  Song_04        2        1

[2421 rows x 4 columns]


In [32]:
# how much track is played per station
df8 = pd.read_sql_query("""
select distinct a.station_id,
                      a.artist,
       a.track,
       count(stamp) OVER (PARTITION BY station_id, artist, track) as listens
from (select c.*, fsm.station_id
      from consumptions c
      left join feed_station_mapping fsm on c.feed_id = fsm.feed_id) as a
      where station_id is not null
order by station_id asc, listens desc;""",con)
print(df8)

                station_id     artist    track  listens
0      smp_station_name_01  artist_06  Song_03       18
1      smp_station_name_01  artist_02  Song_01       16
2      smp_station_name_01  artist_01  Song_08        1
3      smp_station_name_01  artist_04  Song_07        1
4      smp_station_name_01  artist_07  Song_04        1
...                    ...        ...      ...      ...
2383  smp_station_name_997  artist_07  Song_04       34
2384  smp_station_name_997  artist_03  Song_05       24
2385  smp_station_name_998  artist_08  Song_02        8
2386  smp_station_name_999  artist_02  Song_01       52
2387  smp_station_name_999  artist_07  Song_04       50

[2388 rows x 4 columns]


In [51]:
#BONUS top 2 track per station id
df7 = pd.read_sql_query("""
select station_id,
       artist || ' - ' || track as concat_track,
       listens,
       rank_id
from (select distinct station_id,
                      artist,
       track,
       listens,
       rank() OVER (PARTITION BY station_id ORDER BY listens DESC) as rank_id
from (select distinct a.station_id,
                      a.artist,
       a.track,
       count(stamp) OVER (PARTITION BY station_id, artist, track) as listens
        from (select c.*, fsm.station_id
                from consumptions c
                left join feed_station_mapping fsm on c.feed_id = fsm.feed_id) as a
        where station_id is not null
) as b
group by station_id, artist, track
order by station_id) c
where rank_id <3;""", con)
print(df7)

                station_id         concat_track  listens  rank_id
0      smp_station_name_01  artist_06 - Song_03       18        1
1      smp_station_name_01  artist_02 - Song_01       16        2
2      smp_station_name_02  artist_07 - Song_04       71        1
3      smp_station_name_03  artist_07 - Song_04       31        1
4      smp_station_name_04  artist_03 - Song_05      868        1
...                    ...                  ...      ...      ...
1816  smp_station_name_997  artist_02 - Song_01       73        1
1817  smp_station_name_997  artist_07 - Song_04       34        2
1818  smp_station_name_998  artist_08 - Song_02        8        1
1819  smp_station_name_999  artist_02 - Song_01       52        1
1820  smp_station_name_999  artist_07 - Song_04       50        2

[1821 rows x 4 columns]
